<a href="https://colab.research.google.com/github/WisinYWandel/hello-world/blob/master/cylinder_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:

import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
print(tf.__version__)

1.15.2


In [0]:

# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['Volume','Height','Radius', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./Cylinder_train.csv', header = None, names = CSV_COLUMNS)
df_test = pd.read_csv('./Cylinder_test.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./Cylinder_test.csv', header = None, names = CSV_COLUMNS)

In [0]:
df_test.head()

,Volume,Height,Radius,key
0,5235.803667,22.490878,8.608224,1
1,44.121058,9.775056,1.198639,2
2,3960.837241,39.587564,5.643378,3
3,4540.881745,36.378951,6.303330,4
4,3419.147147,19.748998,7.423546,5


In [0]:

def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size=128,
    num_epochs=100,
    shuffle=True,
    queue_capacity=10000
  )

In [0]:
def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    shuffle=False
  )

In [0]:
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x=df, shuffle=False
  )

In [0]:
feature_columns=[
                 tf.feature_column.numeric_column('Height', dtype=tf.float32),
                 tf.feature_column.numeric_column('Radius', dtype=tf.float32)
]

In [0]:

def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))


In [24]:

OUTDIR='./cylinder_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
      feature_columns = feature_columns, model_dir = OUTDIR)
model.train(input_fn = make_train_input_fn(df_train, num_epochs = 100));
print_rmse(model, df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './cylinder_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff16ea9e518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done

In [27]:
predictions = model.predict(input_fn = make_prediction_input_fn(df_test[10:20]))
for items in predictions:
  print(items)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cylinder_trained/model.ckpt-3907
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([6.121185], dtype=float32)}
{'predictions': array([6.121185], dtype=float32)}
{'predictions': array([6.121185], dtype=float32)}
{'predictions': array([6.121185], dtype=float32)}
{'predictions': array([6.121185], dtype=float32)}
{'predictions': array([6.121185], dtype=float32)}
{'predictions': array([6.121185], dtype=float32)}
{'predictions': array([6.121185], dtype=float32)}
{'predictions': array([6.121185], dtype=float32)}
{'predictions': array([6.121185], dtype=float32)}


In [15]:
print_rmse(model, df_test)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-22T21:10:03Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cylinder_trained/model.ckpt-40
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-05-22-21:10:03
INFO:tensorflow:Saving dict for global step 40: average_loss = 25362.283, global_step = 40, label/mean = 139.74388, loss = 3170285.5, prediction/mean = 0.5557398
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 40: ./cylinder_trained/model.ckpt-40
RMSE on dataset = 159.25540161132812


In [20]:
df_train.describe()

,Volume,Height,Radius,key
count,5000.000000,5000.000000,5000.000000,5000.000000
mean,144.189453,3.512988,3.508630,2500.500000
std,77.921983,0.868211,0.860001,1443.520003
min,26.879301,2.000196,2.001180,1.000000
25%,79.731013,2.752870,2.761899,1250.750000
50%,130.520027,3.527128,3.513592,2500.500000
75%,193.827965,4.267606,4.244133,3750.250000
max,391.168034,4.998284,4.999959,5000.000000
